
# Glue Studio Notebook
You are now running a **Glue Studio** notebook; before you can start using your notebook you *must* start an interactive session.

## Available Magics
|          Magic              |   Type       |                                                                        Description                                                                        |
|-----------------------------|--------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------|
| %%configure                 |  Dictionary  |  A json-formatted dictionary consisting of all configuration parameters for a session. Each parameter can be specified here or through individual magics. |
| %profile                    |  String      |  Specify a profile in your aws configuration to use as the credentials provider.                                                                          |
| %iam_role                   |  String      |  Specify an IAM role to execute your session with.                                                                                                        |
| %region                     |  String      |  Specify the AWS region in which to initialize a session.                                                                                                 |
| %session_id                 |  String      |  Returns the session ID for the running session.                                                                                                          |
| %connections                |  List        |  Specify a comma separated list of connections to use in the session.                                                                                     |
| %additional_python_modules  |  List        |  Comma separated list of pip packages, s3 paths or private pip arguments.                                                                                 |
| %extra_py_files             |  List        |  Comma separated list of additional Python files from S3.                                                                                                 |
| %extra_jars                 |  List        |  Comma separated list of additional Jars to include in the cluster.                                                                                       |
| %number_of_workers          |  Integer     |  The number of workers of a defined worker_type that are allocated when a job runs. worker_type must be set too.                                          |
| %glue_version               |  String      |  The version of Glue to be used by this session. Currently, the only valid options are 2.0 and 3.0 (eg: %glue_version 2.0).                               |
| %security_config            |  String      |  Define a security configuration to be used with this session.                                                                                            |
| %sql                        |  String      |  Run SQL code. All lines after the initial %%sql magic will be passed as part of the SQL code.                                                            |
| %streaming                  |  String      |  Changes the session type to Glue Streaming.                                                                                                              |
| %etl                        |  String      |  Changes the session type to Glue ETL.                                                                                                                    |
| %status                     |              |  Returns the status of the current Glue session including its duration, configuration and executing user / role.                                          |
| %stop_session               |              |  Stops the current session.                                                                                                                               |
| %list_sessions              |              |  Lists all currently running sessions by name and ID.                                                                                                     |
| %worker_type                |  String      |  Standard, G.1X, *or* G.2X. number_of_workers must be set too. Default is G.1X.                                                                           |
| %spark_conf                 |  String      |  Specify custom spark configurations for your session. E.g. %spark_conf spark.serializer=org.apache.spark.serializer.KryoSerializer.                      |

In [3]:
%number_of_workers 8
%worker_type G.2X

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Installed kernel version: 0.37.0 
Previous number of workers: 5
Setting new number of workers to: 8
Previous worker type: G.1X
Setting new worker type to: G.2X


In [1]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job

sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

Authenticating with environment variables and user-defined glue_role_arn: arn:aws:iam::078881089437:role/GlueNotebookRole
Trying to create a Glue session for the kernel.
Worker Type: G.2X
Number of Workers: 8
Session ID: fbb0f648-5813-49f7-9dd5-157215bd8479
Job Type: glueetl
Applying the following default arguments:
--glue_kernel_version 0.37.0
--enable-glue-datacatalog true
Waiting for session fbb0f648-5813-49f7-9dd5-157215bd8479 to get into ready status...
Session fbb0f648-5813-49f7-9dd5-157215bd8479 has been created.



## 1. Read data from Data Catalog

In [4]:
reviews_table = glueContext.create_dynamic_frame.from_catalog(
    database='steam_reviews',
    table_name='reviews'
)

In [5]:
reviews_table.printSchema()

root
|-- app_id: long
|-- app_name: string
|-- review_text: string
|-- review_score: long
|-- review_votes: long


### Convert the Dynamic Frame to a Spark DataFrame

In [6]:
# convert the dynamic dataframe to a Spark dataframe
df = reviews_table.toDF()
df.show(5)

+------+--------------+--------------------+------------+------------+
|app_id|      app_name|         review_text|review_score|review_votes|
+------+--------------+--------------------+------------+------------+
|    10|Counter-Strike|     Ruined my life.|           1|           0|
|    10|Counter-Strike|This will be more...|           1|           1|
|    10|Counter-Strike|This game saved m...|           1|           0|
|    10|Counter-Strike|• Do you like ori...|           1|           0|
|    10|Counter-Strike|        Easy to l...|           1|           1|
+------+--------------+--------------------+------------+------------+
only showing top 5 rows


## 2. Data Processing
### 2.1 Check for null values
Here we check for empty string values and blank cell values in the `review_text` and `app_name` columns.

In [7]:
import pyspark.sql.functions as F
from pyspark.sql.functions import isnan, when, count, col

df.filter((df["review_text"] == "") | (df["review_text"] == ' ') | df["review_text"].isNull() | isnan(df["review_text"])).count()

7419


In [8]:
df.filter((df["app_name"] == "") | (df["app_name"] == ' ') | df["app_name"].isNull() | isnan(df["app_name"])).count()

183234


In [9]:
df.filter((df["review_text"] != "") & (df["review_text"] != ' ') & df["review_text"].isNotNull() & ~isnan(df["review_text"])).count()

6409687


In [10]:
df.filter((df["review_text"] == "") | (df["review_text"] == ' ')).count()

7419


In [11]:
df = df.dropna()

In [12]:
df.count()

6417106


In [13]:
df.filter((df["review_text"] != "") | (df["review_text"] != ' ') | df["review_text"].isNotNull()).count()

6417106


### 2.2 Drop rows that contain empty strings

In [14]:
df = df.filter((df["review_text"] != "") & (df["review_text"] != ' ') & df["review_text"].isNotNull() & ~isnan(df["review_text"]))
df = df.filter((df["app_name"] != "") & (df["app_name"] != ' ') & df["app_name"].isNotNull() & ~isnan(df["app_name"]))

Verify that records with empty strings were dropped.

In [15]:
df.filter((df["review_text"] == "") | (df["review_text"] == ' ') | df["review_text"].isNull() | isnan(df["review_text"])).count()

0


In [16]:
df.filter((df["app_name"] == "") | (df["app_name"] == ' ') | df["app_name"].isNull() | isnan(df["app_name"])).count()

0


In [17]:
df.count()

6226617


### 2.3 Trim whitespace from both ends of the review text
Create a new dataframe called `df_records` and trim whitespace.

In [18]:
from pyspark.sql.functions import trim, lower
df_records = df.withColumn("review_text", trim(df.review_text))

### 2.4 Remove "Early Access Review" records

In [19]:
df_records = df_records.filter(df_records["review_text"] != "Early Access Review")

In [20]:
df_records.count()

5238645


## 3. Reduce data to top 20 games
For our analysis we will focus on the top 20 games based on number of reviews.

In [21]:
top20_games_list = list(df_records.groupby('app_name').count().sort('count', ascending = False).select('app_name').toPandas()['app_name'][:20])

In [22]:
top20_games_list

['PAYDAY 2', 'Terraria', 'Dota 2', 'Rocket League', 'Undertale', 'Left 4 Dead 2', 'Warframe', 'Grand Theft Auto V', 'Portal 2', 'Fallout: New Vegas', 'Arma 3', 'The Witcher 3: Wild Hunt', 'BioShock Infinite', 'DARK SOULS™: Prepare To Die Edition', "Garry's Mod", 'Insurgency', 'Mount & Blade: Warband', 'FTL: Faster Than Light', "No Man's Sky", 'Call of Duty: Black Ops III']


### 3.1 Filter dataframe for top 20 games

In [23]:
df_records = df_records.filter(F.col('app_name').isin(top20_games_list)).sort('app_name')

In [24]:
df_records.count()

846712


### 3.2 Lowercase the review text

In [25]:
df_records = df_records.withColumn("review_text", lower(df_records.review_text))

In [26]:
df_records.show()

+------+--------+--------------------+------------+------------+
|app_id|app_name|         review_text|review_score|review_votes|
+------+--------+--------------------+------------+------------+
|107410|  Arma 3|my first day in a...|           1|           1|
|107410|  Arma 3|if you have frien...|           1|           0|
|107410|  Arma 3|recommending this...|           1|           1|
|107410|  Arma 3|can't tell you if...|           1|           1|
|107410|  Arma 3|i wanna play a zo...|           1|           1|
|107410|  Arma 3|i have owned this...|           1|           1|
|107410|  Arma 3|it's not a good '...|           1|           1|
|107410|  Arma 3|oh man. where to ...|           1|           1|
|107410|  Arma 3|having accumulate...|           1|           1|
|107410|  Arma 3|after some 1,400 ...|           1|           1|
|107410|  Arma 3|this game made me...|           1|           0|
|107410|  Arma 3|this is quite pos...|           1|           0|
|107410|  Arma 3|do not g

## 4. Upload Data
### 4.1 Convert PySpark dataframe to Dynamic Frame prior to uploading

In [27]:
from awsglue.dynamicframe import DynamicFrame

dyf_records = DynamicFrame.fromDF(df_records, glueContext, "nested")

### 4.2 Upload Dynamic Frame to S3

In [28]:
# write down the data in a Dynamic Frame to S3 location. 
glueContext.write_dynamic_frame.from_options(
                        frame = dyf_records, # dynamic frame
                        connection_type="s3", 
                        connection_options = {"path": "s3://siads696-wi23-steam-data/write_down_dyf_to_s3/"}, 
                        format = "csv", # write as a csv
                        format_options={
                            "separator": ","
                            },
                        transformation_ctx = "datasink2")